Importations

In [12]:
import nltk
import numpy as np
import random
import string # to process standard python strings
import warnings
warnings.filterwarnings('ignore')


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Lecture de donnees du fichier

In [14]:
f=open('/content/drive/My Drive/Racisme.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()# converts to lowercase
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


sent tokens words tokens

In [15]:
sent_tokens[:2]
['un chatbot (également appelé talkbot, chatterbot, bot, im bot, agent interactif ou entité conversationnelle artificielle) est un programme informatique ou une intelligence artificielle qui mène une conversation via des méthodes auditives ou textuelles. ',
 '«ces programmes sont souvent conçus pour simuler de manière convaincante comment un humain se comporterait comme un partenaire conversationnel, réussissant ainsi le test de turing.»']
word_tokens[:2]
['a', 'chatbot', '(', 'also', 'known']

['a', 'chatbot', '(', 'also', 'known']

Pre-processing the raw text

In [16]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

Keyword matching

In [17]:
GREETING_INPUTS = ("bonjour", "salut", "quoi de neuf", "salam", "Bienvenu")
GREETING_RESPONSES = ["bonjour", "salut", "quoi de neuf", "salam", "Bienvenu"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

Generating Response

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer


import cosine

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

Response

In [20]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"Desole! je ne vous compremd pas"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

Feeding lines

In [ ]:
flag=True
print("ROBO: je me nomme Robo. je vous repondrai sur vos questions concernant le Racisme. si vous voulez quitter, taper bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='merci' or user_response=='je vous remercie' ):
            flag=False
            print("ROBO: vous etes la bienvenue..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO:  au revoir!..")

ROBO: je me nomme Robo. je vous repondrai sur vos questions concernant le Racisme. si vous voulez quitter, taper bye!
ROBO: bonjour
ROBO: le racisme est une idéologie qui, partant du postulat1 de l'existence de races au sein de l'espèce humaine2, 
considère que certaines catégories de personnes sont intrinsèquement supérieures à d'autres2.
ROBO: le racisme est une idéologie qui, partant du postulat1 de l'existence de races au sein de l'espèce humaine2, 
considère que certaines catégories de personnes sont intrinsèquement supérieures à d'autres2.
ROBO: le petit larousse a deux définitions du racisme, au sens strict du terme, 
comme « idéologie fondée sur la croyance qu'il existe une hiérarchie entre les groupes humains, les « races » ; 
comportement inspiré par cette idéologie », et au sens large du terme, comme « une attitude d’hostilité répétée 
voire systématique à l’égard d’une catégorie déterminée de personnes ».
